# Wikidata Download - Reconcile with no Biblographic Data

This script adds the Wikidata Q number from the Wikdiata API to a TSV file. 

The script makes some assumstions based on looking for authors/poets/writes/ect

This script modifies the TSV file itself in batches, should the script timeout or other error you can rerun it and it will pickup where it left off, always run it on a backup of your orginal data files.

It creates a new column in the file `wikidata_qid` which holds the MARC XML 

In [21]:
import pandas as pd
import requests
import time
import string
import unicodedata

## Config
Set these variables below based on your setup

`path_to_tsv` - the path to the TSV file you want to run it on

`id_column_name` - the name of the column header that contains persons name

`user_agent` - this is the value put into the headers on each request, it is good practice to identifiy your client/project when working with open free APIs

`pause_between_req` - number of seconds to wait between each API call



In [22]:
path_to_tsv = "/Users/m/Downloads/data-tmp/major_literary_prizes-winners_judges.tsv"
id_column_name = "full_name"
id_column_important_data = 'prize_year'
user_agent = 'USER thisismattmiller - Test Script'
pause_between_req = 0


In [23]:
def add_qid(d):



    # if there is already a value skip it
    if 'author_wikidata' in d:
        if type(d['author_wikidata']) == str:        
            # print('Skip',d[id_column_name])
            return d

    # if pd.isnull(d['match_score']) == False:
    #     if d['match_score'] != 'id title match':
    #       if int(float(d['match_score'])) > 3:
    #         print('skipp')
    #         return d
    #       else:
    #         print(d['author_wikidata'])
    #         d['author_wikidata']  = None
    #         d['author_wikidata_label']  = None
    #         d['author_viaf']  = None
    #         d['author_lccn']  = None
    #         d['match_score']  = None
    #         d['match_log']  = None

            
    #         print(pd.isnull(d['author_wikidata']))

    

    # if d[id_column_name] != 'Claudia Roth Pierpont':
    #     return d

    # # if you want to add any logic to only download some records add it here
    # if type(d['author_lccn']) != str:
    #     print('Skip no LCCN')
    #     return d

    # if d['role'] != 'winner':
    #     return d

 
    if d[id_column_name] == 'No Winner':
        return d

    # # first fire a query off for a name match / this is using the autocomplete, not the full search we can use either
    # url = "https://www.wikidata.org/w/api.php"
    # params = {
    #     'action':'wbsearchentities',
    #     'search':d[id_column_name],
    #     'format':'json',
    #     'errorformat':'plaintext',
    #     'language':'en',
    #     'uselang':'en',
    #     'type':'item',
    # }
    # headers = {
    #     'Accept' : 'application/json',
    #     'User-Agent': user_agent
    # }
    # r = requests.get(url, params=params, headers=headers)

    # data = r.json()
    # print("!!!!!",d[id_column_name])
    # print(data)

    # # make a list of the qids to use
    # qids = []
    # for s in data['search']:
    #     qids.append(s['id'])
    
    # use the full search by default
    url = "https://www.wikidata.org/w/api.php"
    params = {
        'action':'query',
        'srsearch':d[id_column_name],
        'format':'json',
        'list':'search',
        'srlimit':'10'
    }
    headers = {
        'Accept' : 'application/json',
        'User-Agent': user_agent
    }
    r = requests.get(url, params=params, headers=headers)

    data = r.json()
    print("!!!!!",d[id_column_name])
    print(data)

    # make a list of the qids to use
    qids = []
    for s in data['query']['search']:
        qids.append(s['title'])

    total_hits = data['query']['searchinfo']['totalhits']


    if len(qids) == 0:
        return d

    # build the SPARQL query we are going to use
    qids_with_quotes = []
    for q in qids:
        qids_with_quotes.append(f'wd:{q}')

    sparql = f"""
        SELECT ?item ?itemLabel ?occupation ?occupationLabel ?birth ?death ?award ?awardLabel ?viaf ?lccn ?education ?educationLabel
        WHERE 
        {{

            VALUES ?item {{ { " ".join(qids_with_quotes)  }}}

            ?item wdt:P31 wd:Q5.
          
            optional{{
              ?item wdt:P106 ?occupation.
            }}
            optional{{
              ?item wdt:P569 ?birth.
            }}
            optional{{
              ?item wdt:P570 ?death.
            }}          
            optional{{
              ?item wdt:P166 ?award.
            }}    
            optional{{
              ?item wdt:P214 ?viaf.
            }}              
            optional{{
              ?item wdt:P244 ?lccn.
            }}
            optional{{
              ?item wdt:P69 ?education.
            }}




            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
    """
    params = {
        'query' : sparql
    }

    headers = {
        'Accept' : 'application/json',
        'User-Agent': user_agent
    }
    url = "https://query.wikidata.org/sparql"

    print(sparql)
    r = requests.get(url, params=params, headers=headers)

    data = r.json()

    people = {}

    # did we get any results
    if len(data['results']['bindings']) > 0:
      for result in data['results']['bindings']:

        qid = result['item']['value'].split('/')[-1]

        if qid not in people:

          people[qid] = {
            'qid':qid,
            'score':0,
            'score_log':[],
            'occupation':[],
            'birth':None,
            'death':None,
            'viaf':None,
            'lccn':None,
            'label':None,
            'award': [],
            'education':[]
          }
        
        if 'occupationLabel' in result:
          people[qid]['occupation'].append(result['occupationLabel']['value'])
          people[qid]['occupation'] = list(set(people[qid]['occupation']))
        if 'awardLabel' in result:
          people[qid]['award'].append(result['awardLabel']['value'])
          people[qid]['award'] = list(set(people[qid]['award']))

        if 'educationLabel' in result:
          people[qid]['education'].append(result['educationLabel']['value'])
          people[qid]['education'] = list(set(people[qid]['education']))


        if 'itemLabel' in result:
          people[qid]['label'] = result['itemLabel']['value']
        if 'birth' in result:
          try:            
            # if it is set to this value it is "20.th century, which is unhelpful in our use"
            if result['birth']['value'] != '2000-01-01T00:00:00Z':
              people[qid]['birth'] = int(result['birth']['value'].split('-')[0])
          except:
            people[qid]['birth'] = None
        if 'death' in result:
          try:
            if result['death']['value'] != '2000-01-01T00:00:00Z':
              people[qid]['death'] = int(result['death']['value'].split('-')[0])
          except:
            people[qid]['death'] = None
        if 'viaf' in result:
          people[qid]['viaf'] = result['viaf']['value']
        if 'lccn' in result:
          people[qid]['lccn'] = result['lccn']['value']


      

    # they must have a writerly occupation to continue
    for p in people:

      has_writerly_occ = False
      
      for occ in ['writer', 'poet', 'novelist', 'short story writer', 'author', 'literary critic', 'journalist', 'biographer', 'historian', 'comics artist', 'playwright']:
        if occ in people[p]['occupation']:
          has_writerly_occ = True
      
      if has_writerly_occ == False:
        people[p]['score'] = -1
        people[p]['score_log'].append("No writerly occ")
        continue
      else:
        people[p]['score'] = people[p]['score'] +  1
        people[p]['score_log'].append("Has writerly occ")
      

      # points for pretty much the same name from the sheet as in wikidata
      if levenshtein(normalize_string(people[p]['label']),normalize_string(d['full_name'])) < 3:
        people[p]['score'] = people[p]['score'] + 1
        people[p]['score_log'].append("Has very similar name")
      elif levenshtein(normalize_string(people[p]['label']),normalize_string(d['full_name'])) >= 3:
        people[p]['score'] = people[p]['score'] - 1
      elif levenshtein(normalize_string(people[p]['label']),normalize_string(d['full_name'])) >= 5:
        people[p]['score'] = people[p]['score'] - 2
      elif levenshtein(normalize_string(people[p]['label']),normalize_string(d['full_name'])) >= 8:
        people[p]['score'] = people[p]['score'] - 3        

      # points for having a VIAF or LCCN
      if people[p]['lccn'] != None or people[p]['viaf'] != None:
        people[p]['score'] = people[p]['score'] + 1
        people[p]['score_log'].append("Has VIAF OR LCCN")
      
      # points for being not dead when the award was granted
      if people[p]['death'] != None:
        try:
          award_year = int(d['prize_year'])
          if award_year <= people[p]['death']:
            people[p]['score'] = people[p]['score'] + 1     
            people[p]['score_log'].append("not dead when awarded")

        except:
          pass
      
      # if they were not born when the award was granted then very bad
      if people[p]['birth'] != None:
        try:
          
          award_year = int(d['prize_year'])
          if award_year <= people[p]['birth']:
            people[p]['score'] = -1
            people[p]['score_log'].append("not alive when awarded = -1")        
        except:
          pass
      
      if len(people[p]['award']) >0 and type(d['prize_name']) == str:
        awards = " ".join(people[p]['award']).lower().replace('fellowship','').replace('prize','').replace('award','')
        if d['prize_name'].lower().replace('fellowship','').replace('prize','').replace('award','') in awards:
          people[p]['score'] = people[p]['score'] + 1
          people[p]['score_log'].append("matched on award")        


      if len(people[p]['education']) > 0  != None and type(d['elite_institution']) == str:
        edu = " ".join(people[p]['education']).lower().replace('university of ','').replace('university','').replace('college','')
        if d['elite_institution'].lower().replace('university of ','').replace('university','').replace('college','') in edu:
          people[p]['score'] = people[p]['score'] + 1
          people[p]['score_log'].append("matched on edu")        

      if len(people[p]['education']) > 0  != None and type(d['mfa_degree']) == str:
        edu = " ".join(people[p]['education']).lower().replace('university of ','').replace('university','').replace('college','')
        if d['mfa_degree'].lower().replace('university of ','').replace('university','').replace('college','') in edu:
          people[p]['score'] = people[p]['score'] + 1
          people[p]['score_log'].append("matched on edu")  


      

    print(people)
    best_person_score = 0
    best_person = None
    for p in people:
      if people[p]['score'] > best_person_score:
        best_person_score = people[p]['score']
        best_person = people[p]

    if best_person != None:
      if best_person_score > 1:

        d['author_wikidata'] = best_person['qid']
        d['author_wikidata_label'] = best_person['label']

        if best_person['viaf'] != None:
          d['author_viaf'] = best_person['viaf']
        if best_person['lccn'] != None:
          d['author_lccn'] = best_person['lccn']

        d['match_score'] = best_person_score
        d['match_log'] = "|".join(best_person['score_log'])

    time.sleep(pause_between_req)

    return d

def normalize_string(s):
    s = str(s)
    s = s.translate(str.maketrans('', '', string.punctuation))
    s = " ".join(s.split())
    s = s.lower()
    s = s.casefold()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    s = s.replace('the','')
    return s

def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]


In [24]:
# load the tsv
df = pd.read_csv(path_to_tsv, sep='\t', header=0, low_memory=False)


# we are going to split the dataframe into chunks so we can save our progress as we go but don't want to save the entire file on on every record operation
n = 100  #chunk row size
list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]

# loop through each chunk
for idx, df_chunk in enumerate(list_df):

    print("Working on chunk ", idx, 'of', len(list_df))
    list_df[idx] = list_df[idx].apply(lambda d: add_qid(d),axis=1 )  
    

    reformed_df = pd.concat(list_df)
    reformed_df.to_csv(path_to_tsv, sep='\t')




Working on chunk  0 of 72
!!!!! Inge Judd
{'batchcomplete': '', 'query': {'searchinfo': {'totalhits': 0}, 'search': []}}
!!!!! Inge Judd
{'batchcomplete': '', 'query': {'searchinfo': {'totalhits': 0}, 'search': []}}
!!!!! Inge Judd
{'batchcomplete': '', 'query': {'searchinfo': {'totalhits': 0}, 'search': []}}
!!!!! Reuben Bercovitch
{'batchcomplete': '', 'query': {'searchinfo': {'totalhits': 0}, 'search': []}}
!!!!! Pauline Hanson
{'batchcomplete': '', 'continue': {'sroffset': 10, 'continue': '-||'}, 'query': {'searchinfo': {'totalhits': 33}, 'search': [{'ns': 0, 'title': 'Q466220', 'pageid': 439721, 'size': None, 'wordcount': 0, 'snippet': 'Australian politician', 'timestamp': '2023-02-09T10:14:37Z'}, {'ns': 0, 'title': 'Q94510204', 'pageid': 93505444, 'size': None, 'wordcount': 0, 'snippet': '', 'timestamp': '2021-04-12T02:28:13Z'}, {'ns': 0, 'title': 'Q110309825', 'pageid': 105373454, 'size': None, 'wordcount': 0, 'snippet': 'New Zealand tennis player', 'timestamp': '2022-12-13T20:0